In [0]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import numpy as np
from os import listdir

In [0]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

# return a list of tokens without punctuation marks
def pre_processing(sentences):
    tokens = []
    for s in sentences:
        doc = nlp(s)
        for t in doc:
            if t.is_punct == False:
                tokens.append(t.lower_)
    return tokens

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/C13550/Lesson07/Exercise02-03/

/content/drive/My Drive/C13550/Lesson07/Exercise02-03


In [0]:
filename = 'utils/glove.6B.50d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [0]:
intent_route = 'utils/training/'
response_route = 'utils/responses/'

intents = listdir(intent_route)
responses = listdir(response_route)

In [0]:
def doc_vector(tokens):
    feature_vec = np.zeros((50,), dtype="float32")
    for t in tokens:
         feature_vec = np.add(feature_vec, model[t])
    return np.array([np.divide(feature_vec,len(tokens))])

In [8]:
doc_vectors = np.empty((0,50), dtype='f')
for i in intents:
    with open(intent_route + i) as f:
        sentences = f.readlines()
    sentences = [x.strip() for x in sentences]
    sentences = pre_processing(sentences)
    # adding the document vector to the array doc_vectors
    doc_vectors=np.append(doc_vectors,doc_vector(sentences),axis=0)
print("Vector representation of each document:\n",doc_vectors)

Vector representation of each document:
 [[ 0.14770724  0.5464866  -0.16015846 -0.24245347  0.4202002  -0.2871844
  -0.5529633   0.30648416 -0.26304525  0.18462633 -0.37964222  0.19630247
  -0.21136111  0.12203988  0.75448555  0.05365466 -0.05886668 -0.10289577
  -0.3150087  -0.45763    -0.20689546  0.3401133   0.29135    -0.04262844
   0.5196933  -1.4245243  -0.69796884  0.24098887  0.38095883 -0.5922744
   2.7514176   0.62179    -0.4912156   0.33274615 -0.03507556 -0.10477156
   0.31766444  0.00739434  0.03692211 -0.5429275  -0.12317456  0.00449314
  -0.32877222  0.06327023  0.24757276  0.11004833  0.02727033 -0.14340179
  -0.13737655  0.21054223]
 [ 0.4196458   0.0582203   0.070118   -0.19546     0.5108501  -0.11875646
  -0.42002162 -0.13416906 -0.2159363   0.05899461 -0.18535121  0.418683
  -0.3417599   0.0096156   0.76261395  0.588184    0.431321    0.10539579
   0.062165   -0.77512705 -0.25137132  0.27534202  0.58613455  0.3648346
   0.562019   -1.6857541  -0.71831906  0.32524487

In [0]:
def select_intent(sent_vector, doc_vector):
    index = -1
    similarity = -1 #cosine_similarity is in the range of -1 to 1
    for idx,v in zip(range(len(doc_vector)),doc_vector):
        v = v.reshape(1,-1)
        sent_vector = sent_vector.reshape(1,-1)
        aux = cosine_similarity(sent_vector, v).reshape(1,)
        if aux[0] > similarity:
            index = idx
            similarity = aux
    return index

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
user_sentence = "How are you"

user_sentence = pre_processing([user_sentence])
user_vector = doc_vector(user_sentence).reshape(50,)
intent = intents[select_intent(user_vector, doc_vectors)]
intent

'how_are_you.txt'

In [14]:
def send_response(intent_name):
    with open(response_route + intent_name) as f:
        sentences = f.readlines()
    sentences = [x.strip() for x in sentences]
    return sentences[np.random.randint(low=0, high=len(sentences)-1)]
send_response(intent)

"I'm good!"